In [ ]:
from common.enums import Direction
from simulation.sim_types import TokenPair

import pandas as pd

In [ ]:
l = pd.read_csv('data/stellaswap_metadata_snapshot.csv')
l = l[l.pair_address.isin(pd.read_csv('data/stellaswap_liquid_pairs.csv').pair_address)]
l

In [ ]:
max_our_trade_impact_rate_bps = 15
lst = []
for _, row in l.iterrows():
    pair = TokenPair(row.pair_address, row.token0_address, row.token1_address)
    pair.set_reserves(int(row.reserve0), int(row.reserve1))
    cur_quote = pair.quote_no_fees(Direction.FORWARD)
    _, token1_amount, token0_amount = pair.compute_trade_for_target_forward_quote(cur_quote * (1 + max_our_trade_impact_rate_bps / 10_000))
    adj_token0_amount, adj_token1_amount = token0_amount / 10**row.token0_decimals, token1_amount / 10**row.token1_decimals
    if row.token0_symbol == 'WGLMR':
        usd_value = adj_token0_amount * 0.5 # approx, varies with time
    elif row.token1_symbol == 'WGLMR':
        usd_value = adj_token1_amount * 0.5
    elif row.token0_symbol == 'USDC':
        usd_value = adj_token0_amount
    elif row.token1_symbol == 'USDC':
        usd_value = adj_token1_amount
    lst.append((usd_value, (f'{row.symbol}: ${usd_value:0.2f} -> {row.token0_symbol} = {adj_token0_amount}, {row.token1_symbol} = {adj_token1_amount}')))

for _, v in sorted(lst, reverse=True):
    print(v)

In [ ]:
max_our_trade_impact_rate_bps = 0
lst = []
for _, row in l.iterrows():
    pair = TokenPair(row.pair_address, row.token0_address, row.token1_address)
    pair.set_reserves(int(row.reserve0), int(row.reserve1))
    cur_quote = pair.quote_no_fees(Direction.FORWARD)
    _, token1_amount, token0_amount = pair.compute_trade_for_target_forward_quote(cur_quote * (1 + max_our_trade_impact_rate_bps / 10_000))
    adj_token0_amount, adj_token1_amount = token0_amount / 10**row.token0_decimals, token1_amount / 10**row.token1_decimals
    adj_token0_amount = pair.reserve0 / 10**row.token0_decimals
    adj_token1_amount = pair.reserve1 / 10**row.token1_decimals
    if row.token0_symbol == 'WGLMR':
        usd_value = adj_token0_amount * 0.5 # approx, varies with time
    elif row.token1_symbol == 'WGLMR':
        usd_value = adj_token1_amount * 0.5
    elif row.token0_symbol == 'USDC':
        usd_value = adj_token0_amount
    elif row.token1_symbol == 'USDC':
        usd_value = adj_token1_amount
    lst.append((usd_value, f'{row.symbol}: ${usd_value:0.2f} -> {row.token0_symbol} = {adj_token0_amount}, {row.token1_symbol} = {adj_token1_amount}'))

for v in sorted(lst, reverse=True):
    print(v)